## Humpback Whale challenge

Proposed solution to solve [this challenge](https://www.kaggle.com/c/humpback-whale-identification).

Prithvi Raju [email](nprihviraj24@gmail.com)

<strong>Objective<strong>: identify individual whales in images. 
    
Constituents of dataset: 

    - train.zip - a folder containing the training images
    - train.csv - maps the training Image to the appropriate whale Id. Whales that are not predicted to have a label identified in the training data should be labeled as new_whale.
    - test.zip - a folder containing the test images to predict the whale Id
    - sample_submission.csv - a sample submission file in the correct format
    
    

### Evaluating the data

Before building the model, lets understand the data first. 

In [25]:
import pandas as pd
file = pd.read_csv('train.csv')
ids = file.Id
print(" Number of training samples: ", ids.shape)
uni = file.Id.value_counts()
gt1 = uni[uni>1]
print(" Number of unique whales with pictures more than one: ", gt1)


 Number of training samples:  (9850,)
 Number of unique whales with pictures more than one:  new_whale    810
w_1287fbc     34
w_98baff9     27
w_7554f44     26
w_1eafe46     23
            ... 
w_80c692d      2
w_eb44149      2
w_73cbacd      2
w_17a3581      2
w_0466071      2
Name: Id, Length: 2031, dtype: int64


In [28]:
import os
path, dirs, files = next(os.walk("train/train"))
file_count = len(files)
print("Number of images in train folder: ", file_count)

file.loc[1]

Number of images in train folder:  9850


Image    000466c4.jpg
Id          w_1287fbc
Name: 1, dtype: object

We can conclude that there are 4251 classes for only 9850 images. Most of the "class" have only one training image.

## Few-shot learning using CNN block from Siamese network and  Batch hard strategy for optimization

<br />

<h2> Pipeline to create a maching network<h2>

>  Data preprocessing and augmentation 

    Through this post from he has done data preprocessing steps

>  Matching Network
    
    A method used to represent discrete variables as continuous vectors.
   
>> Build an Encoder Network  

>> Generate "image" embeddings 

>> Pairwise distance between query samples and support sets.

>> Calculating predictions by taking weighted average of the support set labels with the normalised distance.


>  Batch hard strategy for addressing loss functions 

            By using Online triplet mining.




Building the model, I incorporate the ResNet block, wonderfully explained in this [blog](http://teleported.in/posts/decoding-resnet-architecture/) post. 

Why ResNet?

The rule of thumb suggests that ResNet has has worked in this case.

`` The idea is to form a subblock with a 1x1 convolution reducing the number of features, a 3x3 convolution and another 1x1 convolution to restore the number of features to the original. The output of these convolutions is then added to the original tensor (bypass connection). I use 4 such subblocks by block, plus a single 1x1 convolution to increase the feature count after each pooling layer. ``



### PyTorch model creation

The branch model is composed of 6 blocks, each block processing maps with smaller and smaller resolution,, with intermediate pooling layers.

    Block 1 - 384x384
    Block 2 - 96x96
    Block 3 - 48x48
    Block 4 - 24x24
    Block 5 - 12x12
    Block 6 - 6x6

> Block 1 has a single convolution layer with stride 2 followed by 2x2 max pooling. Because of the high resolution, it uses a lot of memory, so a minimum of work is done here to save memory for subsequent blocks.

> Block 2 has two 3x3 convolutions similar to VGG. These convolutions are less memory intensive then the subsequent ResNet blocks, and are used to save memory. Note that after this, the tensor has dimension 96x96x64, the same volume as the initial 384x384x1 image, thus we can assume no significant information has been lost.

> Blocks 3 to 6 perform ResNet like convolution. I suggest reading the original paper, but the idea is to form a subblock with a 1x1 convolution reducing the number of features, a 3x3 convolution and another 1x1 convolution to restore the number of features to the original. The output of these convolutions is then added to the original tensor (bypass connection). I use 4 such subblocks by block, plus a single 1x1 convolution to increase the feature count after each pooling layer.

> The final step of the branch model is a global max pooling, which makes the model robust to fluke not being always well centered.


I'm acknowledging this [notebook](https://www.kaggle.com/martinpiotte/whale-recognition-model-with-score-0-78563) because I incorporate few preprocessing techniques to make it work. When it comes to data preprocessing, I look what rule of thumb suggests because they make life so much easier. 

### Other interesting methods:

<br />

#### Generic One shot siamese network.
- Learning a vector representation of a complex input, like an image, is an example of dimensionality reduction. 
- Taking <strong> Contrastive loss </strong> where Distance between two embeddings of similar class are optimized by bringing it closer, and .


## Defining the Encoder Network Architecture 

I've tried to keep everything succinct and detailed. 

An encoder CNN architecture. Instead of an MLP, which used linear, fully-connected layers, we can use:
* [Convolutional layers](https://pytorch.org/docs/stable/nn.html#conv2d), which can be thought of as stack of filtered images.
* [Maxpooling layers](https://pytorch.org/docs/stable/nn.html#maxpool2d), which reduce the x-y size of an input, keeping only the most _active_ pixels from the previous layer.
* The usual Linear + Dropout layers to avoid overfitting and produce a 10-dim output.
* Batch Normalization layer: The motivation behind it is purely statistical: it has been shown that normalized data, i.e., data with zero mean and unit variance, allows networks to converge much faster. So we want to normalize our mini-batch data, but, after applying a convolution, our data may not still have a zero mean and unit variance anymore. So we apply this batch normalization after each convolutional layer.

### What about selecting the right kernel size?
We always prefer to use smaller filters, like 3×3 or 5×5 or 7×7, but which ones of theses works the best? 

<br />


#### Now it is seldom used in practice to create your own encoder network uniquely from scratch because there are so many  architecture  that will do the job. And these architectures are implemented in different frameworks.

Example: 

ResNet18, VGG-16 etc. Slight modification to these networks will do the job. 



In [5]:
import torchvision
import torch.utils.data as utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
import os
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision.utils

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        
        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            
            nn.Conv2d(1, 96, kernel_size=11,stride=1),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(3, stride=2),
            
            nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),

            nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),

        )
        
        # Defining the fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(30976, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
            
            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),
            
            nn.Linear(128,2))
        
    def forward_once(self, x):
        # Forward pass 
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # forward pass of input  as embedding
        output1 = self.forward_once(input1)
        return output1


Please Note: This notebook only briefs about how I've build the model which is yet to be tested. Theoretically, this model should solve the problem optimally. 

### Approach to the solution:
    
To train our model, we need to first make it learn which class does a one belong to. 
       
     - Initially we take an image (class/label Z), call it as anchor image. After encoding, we represent this image somewhere in Euclidean space with D dimensions, let's assume the location is A.
     - We take the another image of same class Z, call it as positive. We represent this image somewhere in the same Euclidean space with D dimensions,  say B.
     - A different third image is picked with different class, say Y, and call it as negative, represent in same space at point C. The below picture captures Anchor, positie and negative beautifully.

Now our objective is to train the model such that <strong>same</strong> class images should be close that different class images. In short, let's consider $d$ as function of distance (generally, $L_2$ because it gives the squared value), then

<center>$d(anchor, negative) > d(anchor, postive)$</center>
and it should be at least by a margin. 

 
    
![Obama](obama.png)

### Why Triplet loss?

[Reference](https://www.youtube.com/watch?v=d2XB5-tuCWU)

Rather than calculating loss based on two examples ( contrastive loss ), triplet loss involves an anchor example and one positive or matching example (same class) and one negative or non-matching example (differing class).

The loss function penalizes the model such that the distance between the matching examples is reduced and the distance between the non-matching examples is increased. Explanation: 
For some distance on the embedding space d, the loss of a triplet (a,p,n) is:
<center> $L=max(d(a,p)−d(a,n)+margin,0)$ </center>

We minimize this loss, which pushes $d(a,p)$ to 0 and $d(a,n)$ to be greater than $d(a,p)+margin$. As soon as n becomes an “easy negative”, the loss becomes zero.



### So how do I build such a loss function optimally?

#### Keyword: Optimally.

Let me first acknowledge this [paper](https://arxiv.org/abs/1703.07737). 

            Images are now referred as embeddings.
            
Before calculating the loss,  we need sample only the relevant triplets (i.e anchor, positive and negative). To explain it much better, let's categorize the triplets in three different categories:
- Easy negative:  The one where $d(negative, anchor) >> d(positive, anchor)$, if this is the case, $L$ will be zero (recall from previous cell). So implicitly, there wont be gradient that will be propagated backwards.
- Hard negative: The case where $d(negative, anchor) < d(positive, anchor.)$. This means that network performed poorly, and there will be a significant gradient calculated to modify the weights (based on optimization).
- Semi-hard: tiplets where the negative is not closer to the anchor than the positive, but which still have positive loss: $d(a,p)<d(a,n)<d(a,p)+margin$

<br />

##### Offline triplet mining

- The first way to produce triplets is to find them offline, at the beginning of each epoch for instance. We compute all the embeddings on the training set, and then only select hard or semi-hard triplets. We can then train one epoch on these triplets.
- Concretely, we would produce a list of triplets (i,j,k). We would then create batches of these triplets of size B, which means we will have to compute 3B embeddings to get the B triplets, compute the loss of these B triplets and then backpropagate into the network.
- Overall this technique is not very efficient since we need to do a full pass on the training set to generate triplets. It also requires to update the offline mined triplets regularly.


##### Online triplet mining

The idea here is to compute useful triplets on the fly, for each batch of inputs. Given a batch of B examples (for instance B images of faces), we compute the B embeddings and we then can find a maximum of B3 triplets. Of course, most of these triplets are not valid (i.e. they don’t have 2 positives and 1 negative).

Suppose that you have a batch of whale flukes as input of size B=PK, composed of P different flukes with K images each. A typical value is K=4
    
. The two strategies are:

   batch all: 
    - select all the valid triplets, and average the loss on the hard and semi-hard triplets.
    - a crucial point here is to not take into account the easy triplets (those with loss 0), as averaging on them would make the overall loss very small this produces a total of PK(K−1)(PK−K) triplets (PK anchors, K−1 possible positives per anchor, PK−K possible negatives)

   batch hard: 
    - for each anchor, select the hardest positive (biggest distance d(a,p)) and the hardest negative among the batch this produces PK triplets 
    - the selected triplets are the hardest among the batch


### Important Note:

As a machine learning practitioner, it is believed in community that one way of implementing an algorithm on a dataset will not always yield a similar result for different dataset. So naturally, we should always explore different options. 

<br />

Since, I prefer to write in PyTorch, there's an equivalent code for implementing batch-hard-strategy <strong>criterion</strong>.

In [ ]:
import torch

## Courtesy: https://github.com/NegatioN/OnlineMiningTripletLoss/

def batch_hard_triplet_loss(labels, embeddings, margin, squared=False, device='cpu'):
    """Build the triplet loss over a batch of embeddings.
    For each anchor, we get the hardest positive and hardest negative to form a triplet.
    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels, device).float()

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = mask_anchor_positive * pairwise_dist

    # shape (batch_size, 1)
    hardest_positive_dist, _ = anchor_positive_dist.max(1, keepdim=True)

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels).float()

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist, _ = pairwise_dist.max(1, keepdim=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist, _ = anchor_negative_dist.min(1, keepdim=True)

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    tl = hardest_positive_dist - hardest_negative_dist + margin
    tl[tl < 0] = 0
    triplet_loss = tl.mean()

    return triplet_loss

def batch_all_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.
    We generate all the valid triplets and average the loss over the positive ones.
    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    anchor_positive_dist = pairwise_dist.unsqueeze(2)
    anchor_negative_dist = pairwise_dist.unsqueeze(1)

    # Compute a 3D tensor of size (batch_size, batch_size, batch_size)
    # triplet_loss[i, j, k] will contain the triplet loss of anchor=i, positive=j, negative=k
    # Uses broadcasting where the 1st argument has shape (batch_size, batch_size, 1)
    # and the 2nd (batch_size, 1, batch_size)
    triplet_loss = anchor_positive_dist - anchor_negative_dist + margin



    # Put to zero the invalid triplets
    # (where label(a) != label(p) or label(n) == label(a) or a == p)
    mask = _get_triplet_mask(labels)
    triplet_loss = mask.float() * triplet_loss

    # Remove negative losses (i.e. the easy triplets)
    triplet_loss[triplet_loss < 0] = 0

    # Count number of positive triplets (where triplet_loss > 0)
    valid_triplets = triplet_loss[triplet_loss > 1e-16]
    num_positive_triplets = valid_triplets.size(0)
    num_valid_triplets = mask.sum()

    fraction_positive_triplets = num_positive_triplets / (num_valid_triplets.float() + 1e-16)

    # Get final mean triplet loss over the positive valid triplets
    triplet_loss = triplet_loss.sum() / (num_positive_triplets + 1e-16)

    return triplet_loss, fraction_positive_triplets

def _pairwise_distances(embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.
    Args:
        embeddings: tensor of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        pairwise_distances: tensor of shape (batch_size, batch_size)
    """
    dot_product = torch.matmul(embeddings, embeddings.t())

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm = torch.diag(dot_product)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = square_norm.unsqueeze(0) - 2.0 * dot_product + square_norm.unsqueeze(1)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances[distances < 0] = 0

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = distances.eq(0).float()
        distances = distances + mask * 1e-16

        distances = (1.0 -mask) * torch.sqrt(distances)

    return distances

def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    """
    # Check that i, j and k are distinct
    indices_equal = torch.eye(labels.size(0)).byte()
    indices_not_equal = ~indices_equal
    i_not_equal_j = indices_not_equal.unsqueeze(2)
    i_not_equal_k = indices_not_equal.unsqueeze(1)
    j_not_equal_k = indices_not_equal.unsqueeze(0)

    distinct_indices = (i_not_equal_j & i_not_equal_k) & j_not_equal_k


    label_equal = labels.unsqueeze(0) == labels.unsqueeze(1)
    i_equal_j = label_equal.unsqueeze(2)
    i_equal_k = label_equal.unsqueeze(1)

    valid_labels = ~i_equal_k & i_equal_j

    return valid_labels & distinct_indices


def _get_anchor_positive_triplet_mask(labels, device):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check that i and j are distinct
    indices_equal = torch.eye(labels.size(0)).byte().to(device)
    indices_not_equal = ~indices_equal

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = labels.unsqueeze(0) == labels.unsqueeze(1)

    return labels_equal & indices_not_equal


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)

    return ~(labels.unsqueeze(0) == labels.unsqueeze(1)